In [1]:
import os
import re
import copy
import pandas as pd
import numpy as np
import Stack

In [2]:
path = "./files/SampleAD-02-AD.HYP"

In [3]:
def getNetNames(path):
    nets = []
    with open(path, 'r') as f:
        for line in f.readlines():
            if line[:4] == "{NET":
                nets.append(line[5:-1])
    return nets

In [4]:
nets = getNetNames(path)
nets

['NetFS2_1',
 'VOUT2',
 'NetCp45_1',
 'NetCp35_1',
 'NetCp27_1',
 'NetCp23_1',
 'NetCp10_1',
 'NetPf2_6',
 'SI_5388_2.5V',
 'LED6',
 'LED5',
 'NetRp14_2',
 'SYSCLK0_P',
 'FLASH_HOLD',
 'ROW<12>',
 'V24_SCL',
 'V24_ALERT',
 'ADC_SCK',
 'ADC_SDATA',
 'ADC_C\\S\\',
 'DAC_SDI',
 'DAC_SCK',
 'DAC_C\\S\\/LD',
 'CCLK',
 'XTALOUT',
 'XTALIN',
 'USB3_VBUS',
 'USB3.0_RESET#',
 'USB3.0_PMODE2',
 'USB3.0_PMODE1',
 'USB3.0_PMODE0',
 'USB3.0_DQ25',
 'USB3.0_DQ21',
 'U3TX_VDDQ',
 'U3RX_VDDQ',
 'SSTX_P',
 'SSTX_N',
 'SS_TX_P',
 'SS_TX_N',
 'SS_RX_P',
 'SS_RX_N',
 'SS_D_P',
 'SS_D_N',
 'OTG_ID',
 'NetRu15_1',
 'NetRu12_1',
 'NetRu11_1',
 'NetRu1_2',
 'NetJ-1_1',
 'NetFBu1_1',
 'NetCu10_1',
 'I2C_SDA_E2PROM',
 'I2C_SCL_E2PROM',
 'DVDD+1.2V',
 'CLKIN_32',
 'AVDD',
 'VMGTVCCAUX',
 'NetRp1_2',
 'CLK_P',
 'DDR3A_VRP',
 'DDR3A_VRN',
 'SYSCLK1_P',
 'SYSCLK1_N',
 'SYSCLK0_N',
 'POTLOS',
 'NetRd27_2',
 'DDR3A_WE#',
 'DDR3A_RESET#',
 'DDR3A_RAS#',
 'DDR3A_ODT',
 'DDR3A_DQ11',
 'DDR3A_DQ10',
 'DDR3A_DQ7',
 'DDR3A

In [5]:
def getLayersInfo(path, N=0):
    """
    读取HYP文件的层叠信息
    :param path: 文件路径
    :param N: 从第N行开始读文件
    :return: 1. layers = {层名:(第几层, 层的材料)}; 2. idx = 层信息结束的行数
    """
    with open(path, 'r') as f:
        layers = {}
        num = 1
        idx = 0
        flag = False
        for i, line in enumerate(f.readlines()[N:]):
            _line = line.strip()
            # 读取结束, 返回层信息
            if _line[:-1] == "}" and flag:
                flag = False
                idx = i + N + 1
                return layers, idx
            # 读到"STACKUP", 开启层信息的读取
            if _line[1: ] == "STACKUP":
                flag = True
                continue

            # 读取层信息
            if flag:
                # _line = line.strip()
                _line = re.split(r'[ ](?![^=]*\")', _line)
                # 读取信号层的信息
                if _line[0][1:] == "PLANE" or _line[0][1:] == "SIGNAL":
                    for info in _line:
                        # 层名
                        if info[0] == 'L':
                            # 去掉结尾的')'
                            if info[-1] == ')':
                                layerName = info[2:-1]
                            else:
                                layerName = info[2:]
                        # 材料
                        if info[0] == 'M':
                            # 去掉结尾的')'\
                            if info[-1] == ')':
                                material = info[2:-1]
                            else:
                                material = info[2:]
                        else:
                            material = 0
                    layerInfo = (num, material)
                    num = num + 1
                    layers.update({layerName: layerInfo})
                # 读取介质层的信息
                if _line[0][1:] == "DIELECTRIC":
                    continue
        return layers, idx

In [6]:
layers = getLayersInfo(path)
layers

({'"Top Layer"': (1, 'COPPER'),
  'G3': (2, 'COPPER'),
  'S1': (3, 'COPPER'),
  'G1': (4, 'COPPER'),
  'P1': (5, 'COPPER'),
  'S3': (6, 'COPPER'),
  'S4': (7, 'COPPER'),
  'P2': (8, 'COPPER'),
  'G2': (9, 'COPPER'),
  'S2': (10, 'COPPER'),
  'G4': (11, 'COPPER'),
  '"Bottom Layer"': (12, 'COPPER')},
 0)

In [7]:
def getDevicesInfo(path, N=0):
    """
    读取HYP文件的设备信息
    :param path: 文件路径
    :param N: 从第N行开始读文件
    :return: 1. devices:["设备名"]; 2. 设备信息到第几行结束
    """
    # 从一行中找到设备信息的正则表达式
    rule = re.compile('REF=[a-zA-Z]*[0-9]*')
    idx = 0
    with open(path, 'r') as f:
        flag = False
        devices = []
        for i, line in enumerate(f.readlines()[N:]):
            if flag and line[:-1] == '}':
                flag = False
                idx = i + N + 1
                return devices, idx
            if line[1:-1] == 'DEVICES':
                flag = True
                continue
            if flag:
                device = rule.findall(line)
                devices.append(device[0][4:])
        return devices, idx

In [8]:
def getSegsAndPins(path, csvPath, N=0):
    """
    找到HYP文件中所有的线段和管脚
    :param path: 文件路径
    :param N: 从第N行开始读文件
    :return: 1. segs:{坐标点1:[(与坐标点1相连的坐标点a, 线宽, 线段所在的层),(...)]}
             2. aPins:{坐标点：(器件名.管脚号，所在网络)}
             3. nPins:{坐标点：(器件名.管脚号，所在网络)} 和 {器件名: [(坐标，所在网络，管脚号)]}
    """
    # pins里包含了所有的有源器件和无源器件
    pins = getActiveAndNegtivePins(csvPath)
    segs = {}
    aPins = {}
    nPins = {}
    flag = False
    with open(path, 'r') as f:
        for i, line in enumerate(f.readlines()[N:]):
            if line[:] == '{END}':
                return segs, aPins, nPins
            if flag and line[:-1] == '}':
                flag = False
                continue
            if line[1:4] == 'NET':
                flag = True
                net = line[5:-1]
                continue 
            if flag:
                # 首尾去空格
                line = line.strip()
                if line[1:4] == 'SEG':
                    _line = re.split(r'[ ](?![^=]*\")', line)
                    coord1 = (_line[1][3:], _line[2][3:])
                    coord2 = (_line[3][3:], _line[4][3:])
                    width = _line[5][2:]
                    layer = _line[6][2:-1]
                    if segs.get(coord1) is None:
                        segs.update({coord1: [(coord2, width, layer)]})
                    else:
                        pointList = segs.get(coord1)
                        pointList.append((coord2, width, layer))
                        segs.update({coord1: pointList})
                    if segs.get(coord2) is None:
                        segs.update({coord2: [(coord1, width, layer)]})
                    else:
                        pointList = segs.get(coord2)
                        pointList.append((coord1, width, layer))
                        segs.update({coord2: pointList})
                    continue
                if line[1:4] == 'PIN':
                    _line = re.split(r'[ ](?![^=]*\")', line)
                    coord = (_line[1][2:], _line[2][2:])
                    # 找到 器件名.管脚
                    device = re.findall(r'[^=]*\.[^\)]*', _line[3])[0]
                    _device = device.split('.')
                    deviceName = _device[0]
                    pinNum = _device[1]
                    attr = pins.get(deviceName)
                    if attr is None or attr is 0:
                        nPins.update({coord: (device, net)})
                        if nPins.get(deviceName) is None:
                            nPins.update({deviceName: [(coord, net, pinNum)]})
                        else:
                            pointList = nPins.get(deviceName)
                            pointList.append((coord, net, pinNum))
                            nPins.update({deviceName: pointList})
                    else:
                        aPins.update({coord: (device, net)})
                    continue
        return segs, aPins, nPins

In [9]:
def getActiveAndNegtivePins(csvPath):
    """
    找出电路板上所有的pins，并用0/1表示为有源器件或无源器件
    目前一个器件的管脚数量大于2则判断为有源器件
    :param csvPath: 从AD中导出的csv文件
    :return: pins ： {器件名：0/1} （0表示无源，1表示有源）
    """
    pins = {}
    df = pd.read_csv(csvPath, encoding="GBK")
    nDf = df[df["Pins"] <= 2]
    aDf = df[df["Pins"] > 2]
    nPinList = nDf["Designator"]
    aPinList = aDf["Designator"]
    for nPin in nPinList:
        tempList = nPin.split(",")
        for e in tempList:
            pins.update({e:0})
    for aPin in aPinList:
        tempList = aPin.split(",")
        for e in tempList:
            pins.update({e:1})
    return pins

def generateModels(activePins, negtivePins, segs, planeNames):
    keys = activePins.keys()
    models = []
    # 确定dfs的起点,必从有源器件开始
    # TODO
    # (这里是有瑕疵的，但是我懒得改了，应该是从有源output出发, 到input结束最合理)
    # 但是我们看文件不可能知道哪里是output，哪里是input，所以这个问题只能留给后人处理了，问题对结果影响不大
    for key in keys:
        # 如果有源器件的管脚没有连接，则继续
        if segs.get(key) is None or len(segs.get(key)) == 0:
            continue
        else:
            temp = []
            plane = []
            # 信号层
            signals = []
            _models = []
            # (坐标， 器件名.管脚号)
            unit = (key, activePins.get(key)[0])
            # 找到一个所有连接到的plane层的线路
            # 确定起点
            plane.append(unit)
            temp.append(unit)
            plane = findVCCGND(plane, key, segs, activePins, negtivePins, planeNames)
            signals = findAPin(signals, temp, key, segs, activePins, negtivePins)
            _models = genModel(signals, plane)
            tempModels = copy.deepcopy(_models)
            models.append(tempModels)
    return models

In [10]:
def findVCCGND(model, key, segs, aPins, nPins, planeNameList):
    """
    从一个点出发，找到与这个点连接的所有终点为Plane层的线路，实际上连接一个就ok了，不然没意义,此步判断是否有端接
    :param model: 每次迭代保持更新的线路
    :param key: 起点坐标， 一个tuple
    :param segs: {坐标点1:[(与坐标点1相连的坐标点a, 线宽, 线段所在的层),(...)]}
    :param aPins: {坐标点：(器件名.管脚号，所在网络)}
    :param nPins: {坐标点：(器件名.管脚号，所在网络)} 和 {器件名: [(坐标，所在网络，管脚号)]}
    :param planeNameList: plane层在这个电路中的名字列表
    :return:
    """
    segList = segs.get(key)
    if segList == None:
        return None
    for seg in segList:
        # 防止seg重复, 保证每条路不回头
        tempList = segs.get(seg[0])
        for i, elem in enumerate(tempList):
            if elem[0] == key:
                tempList.pop(i)
                segs.update({seg[0]: tempList})
                break
        # 先存储线段：（（左，右），宽度， 层，’seg'）
        unit = ((key, seg[0]), seg[1], seg[2], 'seg')
        model.append(unit)
        aPin = aPins.get(seg[0])
        nPin = nPins.get(seg[0])
        if aPin is not None:
            net = aPin[1]
            if net in planeNameList:
                model.append((seg[0], ('net', net), aPin[0], 'aPin'))
                # 一条SI模型中只允许一个接地或电源，不然没意义
                return model
        if nPin is not None:
            net = nPin[1]
            if net in planeNameList:
                model.append((seg[0], ('net', net), nPin[0], 'nPin'))
                return model
            else:
                # (坐标， 无源器件名.管脚号)
                unit = (seg[0], nPin[0], 'nPin')
                model.append(unit)
                name = nPin[0].split('.')[0]
                _nPins = nPins.get(name)
                for nPin in _nPins:
                    if nPin[0] == seg[0]:
                        continue
                    net = nPin[1]
                    unit = ((seg[0], nPin[0]), seg[1], seg[2], 'seg')
                    model.append(unit)
                    if net in planeNameList:
                        # （坐标，网络名，无源器件名.管脚号）
                        model.append((nPin[0], ('net', net), name+'.'+str(nPin[2]), 'nPin'))
                        return model
                    else:
                        unit = (nPin[0], name+'.'+str(nPin[2]), 'nPin')
                        model.append(unit)
                        _model = findVCCGND(model, nPin[0], segs, aPins, nPins, planeNameList)
                        if _model is not None:
                            return _model
                        model.pop(len(model) - 1)
                    model.pop(len(model) - 1)
                model.pop(len(model) - 1)
        _model = findVCCGND(model, seg[0], segs, aPins, nPins, planeNameList)
        if _model is not None:
            return _model
        model.pop(len(model) - 1)
    return None

In [11]:
def sortModels(activePins, negtivePins, segs, planeNames):
    keys = activePins.keys()
    # 一个model应该有有源pin，端接情况（端接电阻，端接电容，接地电压），线段
    models = []
    for key in keys:
        net = activePins.get(key)[1]
        if net in planeNames:
            continue
        if segs.get(key) is None or len(segs.get(key)) == 0:
            continue
        else:
            temp = []
            plane = []
            signals = []
            unit = (key, activePins.get(key)[0], 'aPin')
            temp.append(unit)
            # plane = findVCCGND(plane, key, segs, activePins, negtivePins, planeNames)
            plane = findVCCAndGNDPath(key, plane, activePins, negtivePins, segs, planeNames)
            # [端接无源器件名，端接的网络名]
            terminateInfo = []
            # 有端接
            if plane is not None:
                for ele in plane:
                    if ele[-1] == 'nPin':
                        terminateInfo.append(ele[-2].split('.')[0])
                        if ele[-3][0] == 'net':
                            terminateInfo.append(ele[-3][1])
                terminateInfo = list(set(terminateInfo))
            signals = findAPinPath(key, temp, activePins, negtivePins, segs)
            for model in signals:
                aPinInfo = []
                segsInfo = []
                for ele in model:
                    if ele[-1] == 'seg':
                        segsInfo.append(ele)
                    else:
                        # Attention: 在aPinInfo里可能存在nPin，在最后的生成文件数据的时候要排查出来加入端接的信息
                        aPinInfo.append(ele)
                models.append([aPinInfo, segsInfo, terminateInfo])
    # models: [[aPins, segs, 端接情况],...]
    return models

In [12]:
def genModel(signals, plane):
    """
    内部函数，主要用来整合到有源pin和到plane层线路，防止重复
    :param signals: 到有源pin的线路列表
    :param plane: 到plane层的线路列表
    :return: 整合好的SI高速模型
    """
    models = []
    if plane is None:
        for elem in signals:
            if elem is None:
                continue
            else:
                models.append(elem)
        return models
    for elem in signals:
        if elem is None:
            continue
        elem = elem + plane
        elem = list(set(elem))
        models.append(elem)
    return models

In [13]:
def segsBi2Stra(key, seg, segs):
    seg_list = segs.get(seg[0])
    for i, elem in enumerate(seg_list):
        if elem[0] == key:
            seg_list.pop(i)
            segs.update({seg[0]: seg_list})

In [14]:
def findAllAPinPath(segs, aPins, nPins):
    models = []
    model = []
    stack = Stack.Stack()
    for key in aPins.keys():
        aPin_unit = (key, aPins.get(key)[0], 'aPin')
        model.append(aPin_unit)
        level = 0
        state = {}
        seg_list = segs.get(key)
        if seg_list is None:
            continue

        if len(seg_list) > 1:
            for seg in seg_list:
                stack.push((seg, key, level))
                segsBi2Stra(key, seg, segs)
            state.update({level: copy.deepcopy(model)})
            level += 1
        elif len(seg_list) == 1:
            stack.push((seg_list[0], key, level))
            segsBi2Stra(key, seg_list[0], segs)
            state.update({level: copy.deepcopy(model)})

        while not stack.isEmpty():
            item = stack.pop()
            model = copy.deepcopy(state.get(item[-1]))
            left = item[1]
            seg = item[0]
            seg_unit = ((left, seg[0]), seg[1], seg[2], 'seg')
            model.append(seg_unit)
            seg_list = segs.get(seg[0])
            aPin = aPins.get(seg[0])
            nPin = nPins.get(seg[0])

            if seg_list is None:
                havaSegs = False
            elif len(seg_list) <= 0:
                havaSegs = False
            else:
                havaSegs = True

            # this point only connect to other segs
            if aPin is None and nPin is None and havaSegs:
                if len(seg_list) > 1:
                    state.update({level: copy.deepcopy(model)})
                    for next_seg in seg_list:
                        stack.push((next_seg, seg[0], level))
                        segsBi2Stra(seg[0], next_seg, segs)
                    level += 1
                else:
                    state.update({level: copy.deepcopy(model)})
                    stack.push((seg_list[0], seg[0], level))
                    segsBi2Stra(seg[0], seg_list[0], segs)

            # this point is only aPin
            elif aPin is not None and nPin is None and not havaSegs:
                aPin_unit = (seg[0], aPin[0], 'aPin')
                model.append(aPin_unit)
                models.append(copy.deepcopy(model))

            # this point is only nPin
            elif aPin is None and nPin is not None and not havaSegs:
                nPin_unit = (seg[0], nPin[0], 'nPin')
                model.append(nPin_unit)
                name = nPin[0].split('.')[0]
                nPin_list = nPins.get(name)
                for nPin_right in nPin_list:
                    if nPin_right[0] == seg[0]:
                        continue
                    seg_unit = ((seg[0], nPin_right[0]), seg[1], seg[2], 'seg')
                    model.append(seg_unit)
                    nPin_unit = (nPin_right[0], name+'.'+str(nPin_right[1]), 'nPin')
                    model.append(nPin_unit)
                    state.update({level: copy.deepcopy(model)})
                    seg_temp = (nPin_right[0], seg[1], seg[2])
                    if nPins.get(nPin_right[0]) is not None:
                        nPins.pop(nPin_right[0])
                    stack.push((seg_temp, seg[0], level))

            # # this point is aPin and connect to other segs
            elif aPin is not None and nPin is None and havaSegs:
                aPin_unit = (seg[0], aPin[0], 'aPin')
                model.append(aPin_unit)
                models.append(copy.deepcopy(model))
                model.pop(len(model) - 1)
                if len(seg_list) > 1:
                    state.update({level: copy.deepcopy(model)})
                    for next_seg in seg_list:
                        stack.push((next_seg, seg[0], level))
                        segsBi2Stra(seg[0], next_seg, segs)
                    level += 1
                else:
                    state.update({level: copy.deepcopy(model)})
                    stack.push((seg_list[0], seg[0], level))
                    segsBi2Stra(seg[0], seg_list[0], segs)

            # # this point is nPin and connect to other segs
            # elif aPin is None and nPin is not None and havaSegs:
            #     # this method is hard to accomplish with stack, and it's really rare to draw pcb like this
            #     # so we ignore the nPin
            #     state.update({level: copy.deepcopy(model)})
            #     name = nPin[0].split('.')[0]
            #     nPin_list = nPins.get(name)
            #     for nPin_right in nPin_list:
            #         if nPin_right[0] == seg[0]:
            #             continue
            #         seg_temp = (nPin_right[0], seg[1], seg[2])
            #         if nPins.get(nPin_right[0]) is not None:
            #             nPins.pop(nPin_right[0])
            #         stack.push((seg_temp, seg[0], level))
            #
            #     for next_seg in seg_list:
            #         stack.push((next_seg, seg[0], level))
            #         segsBi2Stra(seg[0], next_seg, segs)
            #
            #     level += 1

            # this point is nPin, aPin and (or not) connect to other segs
            # or this point is nothing
            else:
                # because no one will draw pcb like this, we drop this point
                continue
        model.clear()
    return models

In [15]:
def findAPinPath(key, model, aPins, nPins, segs):
    models = []
    stack = Stack.Stack()
    level = 0
    state = {}
    seg_list = segs.get(key)
    if seg_list is None:
        return models

    if len(seg_list) > 1:
        for seg in seg_list:
            stack.push((seg, key, level))
            segsBi2Stra(key, seg, segs)
        state.update({level: copy.deepcopy(model)})
        level += 1
    elif len(seg_list) == 1:
        stack.push((seg_list[0], key, level))
        segsBi2Stra(key, seg_list[0], segs)
        state.update({level: copy.deepcopy(model)})
    else:
        return models

    while not stack.isEmpty():
        item = stack.pop()
        model = copy.deepcopy(state.get(item[-1]))
        left = item[1]
        seg = item[0]
        seg_unit = ((left, seg[0]), seg[1], seg[2], 'seg')
        model.append(seg_unit)
        seg_list = segs.get(seg[0])
        aPin = aPins.get(seg[0])
        nPin = nPins.get(seg[0])
        if seg_list is None:
            havaSegs = False
        elif len(seg_list) <= 0:
            havaSegs = False
        else:
            havaSegs = True

        # this point only connect to other segs
        if aPin is None and nPin is None and havaSegs:
            if len(seg_list) > 1:
                state.update({level: copy.deepcopy(model)})
                for next_seg in seg_list:
                    stack.push((next_seg, seg[0], level))
                    segsBi2Stra(seg[0], next_seg, segs)
                level += 1
            else:
                state.update({level: copy.deepcopy(model)})
                stack.push((seg_list[0], seg[0], level))
                segsBi2Stra(seg[0], seg_list[0], segs)

        # this point is only aPin
        elif aPin is not None and nPin is None and not havaSegs:
            aPin_unit = (seg[0], aPin[0], 'aPin')
            model.append(aPin_unit)
            models.append(copy.deepcopy(model))

        # # this point is only nPin
        elif aPin is None and nPin is not None and not havaSegs:
            nPin_unit = (seg[0], nPin[0], 'nPin')
            model.append(nPin_unit)
            name = nPin[0].split('.')[0]
            nPin_list = nPins.get(name)
            for nPin_right in nPin_list:
                if nPin_right[0] == seg[0]:
                    continue
                seg_unit = ((seg[0], nPin_right[0]), seg[1], seg[2], 'seg')
                model.append(seg_unit)
                nPin_unit = (nPin_right[0], name+'.'+str(nPin_right[2]), 'nPin')
                model.append(nPin_unit)
                state.update({level: copy.deepcopy(model)})
                seg_temp = (nPin_right[0], seg[1], seg[2])
                if nPins.get(nPin_right[0]) is not None:
                    nPins.pop(nPin_right[0])
                stack.push((seg_temp, seg[0], level))

        # # this point is aPin and connect to other segs
        elif aPin is not None and nPin is None and havaSegs:
            aPin_unit = (seg[0], aPin[0], 'aPin')
            model.append(aPin_unit)
            models.append(copy.deepcopy(model))
            model.pop(len(model) - 1)
            if len(seg_list) > 1:
                state.update({level: copy.deepcopy(model)})
                for next_seg in seg_list:
                    stack.push((next_seg, seg[0], level))
                    segsBi2Stra(seg[0], next_seg, segs)
                level += 1
            else:
                state.update({level: copy.deepcopy(model)})
                stack.push((seg_list[0], seg[0], level))
                segsBi2Stra(seg[0], seg_list[0], segs)

        # # this point is nPin and connect to other segs
        # elif aPin is None and nPin is not None and havaSegs:
        #     # this method is hard to accomplish with stack, and it's really rare to draw pcb like this
        #     # so we ignore the nPin
        #     state.update({level: copy.deepcopy(model)})
        #     name = nPin[0].split('.')[0]
        #     nPin_list = nPins.get(name)
        #     for nPin_right in nPin_list:
        #         if nPin_right[0] == seg[0]:
        #             continue
        #         seg_temp = (nPin_right[0], seg[1], seg[2])
        #         if nPins.get(nPin_right[0]) is not None:
        #             nPins.pop(nPin_right[0])
        #         stack.push((seg_temp, seg[0], level))
        #     for next_seg in seg_list:
        #         stack.push((next_seg, seg[0], level))
        #         segsBi2Stra(seg[0], next_seg, segs)
        #     level += 1

        # this point is nPin, aPin and (or not) connect to other segs
        # or this point is nothing
        else:
            # because no one will draw pcb like this, we drop this point
            continue
    model.clear()
    return models

In [16]:
def findVCCAndGNDPath(key, model, aPins, nPins, segs, planeNameList):
    seg_list = segs.get(key)
    # level = 0
    # state = {}
    stack = Stack.Stack()
    if seg_list is None or len(seg_list) == 0:
        return None

    if len(seg_list) > 1:
        for seg in seg_list:
            stack.push(seg)
            segsBi2Stra(key, seg, segs)
        # state.update({level: copy.deepcopy(model)})
        # level += 1
    elif len(seg_list) == 1:
        stack.push(seg_list[0])
        segsBi2Stra(key, seg_list[0], segs)
        # state.update({level: copy.deepcopy(model)})
    else:
        return None

    while not stack.isEmpty():
        seg = stack.pop()
        # seg = item[0]
        # left = item[1]
        seg_list = segs.get(seg[0])
        aPin = aPins.get(seg[0])
        nPin = nPins.get(seg[0])

        if seg_list is None:
            havaSegs = False
        elif len(seg_list) <= 0:
            havaSegs = False
        else:
            havaSegs = True

        # this point only connect to other segs
        if aPin is None and nPin is None and havaSegs:
            # state.update({level: copy.deepcopy(model)})
            for next_seg in seg_list:
                stack.push(next_seg)
                segsBi2Stra(seg[0], next_seg, segs)
            # level += 1

        # this point is only aPin
        elif aPin is not None and nPin is None and not havaSegs:
            net = aPin[1]
            if net in planeNameList:
                model.append((seg[0], ('net', net), aPin[0], 'aPin'))
                return model

        # # this point is only nPin
        elif aPin is None and nPin is not None and not havaSegs:
            net = nPin[1]
            if net in planeNameList:
                model.append((seg[0], ('net', net), nPin[0], 'nPin'))
                return model
            else:
                name = nPin[0].split('.')[0]
                nPin_list = nPins.get(name)
                for nPin_right in nPin_list:
                    if nPin_right[0] == seg[0]:
                        continue
                    net = nPin_right[1]
                    if net in planeNameList:
                        nPin_unit = (nPin_right[0], ('net', net), name + '.' + str(nPin_right[2]), 'nPin')
                        model.append(nPin_unit)
                        return model
                    seg_temp = (nPin_right[0], seg[1], seg[2])
                    if nPins.get(nPin_right[0]) is not None:
                        nPins.pop(nPin_right[0])
                    stack.push(seg_temp)

        # # this point is aPin and connect to other segs
        elif aPin is not None and nPin is None and havaSegs:
            net = aPin[1]
            if net in planeNameList:
                model.append((seg[0], ('net', net), aPin[0], 'aPin'))
                return model
            else:
                for next_seg in seg_list:
                    stack.push(next_seg)
                    segsBi2Stra(seg[0], next_seg, segs)

        # this point is nPin and connect to other segs
        elif aPin is None and nPin is not None and havaSegs:
            # this method is hard to accomplish with stack, and it's really rare to draw pcb like this
            # so we ignore the nPin
            net = nPin[1]
            if net in planeNameList:
                model.append((seg[0], ('net', net), nPin[0], 'nPin'))
                return model
            else:
                name = nPin[0].split('.')[0]
                nPin_list = nPins.get(name)
                for nPin_right in nPin_list:
                    if nPin_right[0] == seg[0]:
                        continue
                    net = nPin_right[1]
                    if net in planeNameList:
                        nPin_unit = (nPin_right[0], ('net', net), name+'.'+str(nPin_right[2]), nPin)
                        model.append(nPin_unit)
                        return model
                    seg_temp = (nPin_right[0], seg[1], seg[2])
                    if nPins.get(nPin_right[0]) is not None:
                        nPins.pop(nPin_right[0])
                    stack.push(seg_temp)

                for next_seg in seg_list:
                    stack.push(next_seg)
                    segsBi2Stra(seg[0], next_seg, segs)

        # this point is nPin, aPin and (or not) connect to other segs
        # or this point is nothing
        else:
            # because no one will draw pcb like this, we drop this point
            continue

    return None


In [33]:
def readHYP2File(HYPPath, csvPath, outputPath, planeNames):
    """
    :param HYPPath: HYP文件
    :param csvPath: 电路板器件对应的表
    :param outputPath: 训练用data输出的地址
    :param planeNames: 电源层网络名和地层网络名，客户应该提供这个，不然起名五花八门的没法搞
    :return:
    """
    layers, idx = getLayersInfo(HYPPath)
    devices, idx = getDevicesInfo(HYPPath, idx)
    segs, aPins, nPins = getSegsAndPins(HYPPath, csvPath, idx)
    pins = getActiveAndNegtivePins(csvPath)
    models = sortModels(aPins, nPins, segs, planeNames)
    # 对models进行处理， 使数据达到相同维度
    segMaxDim = 50
    pinMaxDim = 2
    datas = []
    for model in models:
        data = []
        nPinNames = []
        aPinsData = []
        segsData = []
        pinDim = 0
        segDim = 0
        res = 0
        capacity = 0
        volt = 0
        for aPin in model[0]:
            if pinDim >= pinMaxDim:
                break
            if aPin[-1] == "aPin":
                # 如果是aPin需要添加aPin的属性
                name = aPin[1]
                # TODO:通过名字和管脚号在这里写入读文件的代码半自动化生成数据
                # 每个pin：输入输出flag，输入电容，输出电阻，信号上升下降时间
                # 最后添加：传输延迟，工作频率
                # 这里先存默认值：输入电容20，输入电阻100，上升下降0以及坐标
                aPinsData.extend([20, 100, 0, aPin[0][0], aPin[0][1]])
                pinDim += 1
            elif aPin[-1] == 'nPin':
                # 保存nPin的情况
                nPinNames.append(aPin[1].split('.')[0])
        while pinDim < pinMaxDim:
            aPinsData.extend([0, 0, 0, 0, 0])
            pinDim += 1
        # aPin输入完成后统一属性有传输延迟0，工作频率133
        aPinsData.extend([0, 133])
        data.extend(aPinsData)
        # 保存线段，最大100条
        segsHead = model[1][0]
        # 坐标，宽度，层的编号（层的材料暂时没加）
        segsData.extend([segsHead[0][0][0], segsHead[0][0][1], segsHead[1], layers.get(segsHead[2])[0]])
        for seg in model[1]:
            if segDim >= segMaxDim:
                break
            segsData.extend([seg[0][1][0], seg[0][1][1], seg[1], layers.get(seg[2])[0]])
            segDim += 1
        while segDim < segMaxDim:
            segsData.extend([0, 0, 0, 0])
            segDim += 1
        data.extend(segsData)
        for nPin in model[2]:
            #TODO:在这里写入端接电阻电容的值by无源器件的名字
            res = 0
            capacity = 0
            #TODO：通过网络名判断此条传输线上的电压大小
            if nPin in planeNames:
                volt = 0
        data.extend([res, capacity, volt])
        _data = copy.deepcopy(data)
        datas.append(_data)
    pd.DataFrame(datas).to_csv(outputPath, index = False, header = False)
    return pd.DataFrame(datas)

In [34]:
hyp5 = "./files/HYP2/demo2.hyp"
csv5 = "./files/HYP2/demo2.csv"
o5 = "./files/HYP2/output/demo2.csv"
pn5 = ['gnd', 'vcc']

hyp6 = "./files/HYP2/mainboard.hyp"
csv6 = "./files/HYP2/mainborad.csv"
o6 = "./files/HYP2/output/mainboard.csv"
pn6 = ['VCC', 'GND']

In [35]:
readHYP2File(hyp6, csv6, o6, pn6)

,0,1,2,3,4,5,6,7,8,9,...,209,210,211,212,213,214,215,216,217,218
0,20,100,0,2.30500,2.37120,20,100,0,2.85000,2.75000,...,0,0,0,0,0,0,0,0,0,0
1,20,100,0,2.30500,2.37120,20,100,0,3.85000,2.75000,...,0,0,0,0,0,0,0,0,0,0
2,20,100,0,2.30500,2.37120,20,100,0,4.83500,1.91600,...,0,0,0,0,0,0,0,0,0,0
3,20,100,0,2.30500,2.37120,20,100,0,5.03500,1.91600,...,0,0,0,0,0,0,0,0,0,0
4,20,100,0,2.30500,2.37120,20,100,0,4.33500,1.91600,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,20,100,0,2.90000,3.30000,20,100,0,2.90000,3.80000,...,0,0,0,0,0,0,0,0,0,0
153,20,100,0,3.00000,3.50000,20,100,0,3.00000,4.00000,...,0,0,0,0,0,0,0,0,0,0
154,20,100,0,3.30000,3.50000,20,100,0,3.30000,4.00000,...,0,0,0,0,0,0,0,0,0,0
155,20,100,0,3.40000,3.80000,20,100,0,3.60000,2.75000,...,0,0,0,0,0,0,0,0,0,0
